## linear Regression
We start this project with a linear regression model. After training this model, it should be able to give accurate predictions(labels), given a set of inputs(features). Take note that the initial list of weights w and the bias, b, can start of as anything. We will initialise it with all its entries as 1.

 We first need to import the relavent modules.

In [ ]:
import numpy as np
import matplotlib as plt
import seaborn
import pprint
import math
import csv
import random
print = pprint.pprint

Suppose we have some datapoints and we wish to fit a best-fit line $\hat{y} = w_1 x_{1} + w_2 x_{2} + \dots + w_m x_{m} + b$. Let's say $\hat{y}_i$ is the predicted output value for each point $(x_{i1}, ..., x_{im})$, given by $\hat{y}_i = w_1 x_{i1} + w_2 x_{i2} + \dots + w_m x_{im} + b$.  
If we let
$$

\mathbf{X} = \begin{bmatrix}
x_{11} & x_{12} & \cdots & x_{1m} & 1 \\
x_{21} & x_{22} & \cdots & x_{2m} & 1 \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
x_{n1} & x_{n2} & \cdots & x_{nm} & 1
\end{bmatrix}
,
\mathbf{w} = \begin{bmatrix}
w_1 \\
w_2 \\
\vdots \\
w_m \\
b
\end{bmatrix}
,
\mathbf{Y} = \begin{bmatrix}
\hat{y}_1 \\
\hat{y}_2 \\
\vdots \\
\hat{y}_n
\end{bmatrix}
$$

we can write

$$
\mathbf{Y} = \mathbf{X} \mathbf{w}
$$

Now, that's a dirty little trick.

Suppose we fit line, how then do we know how "good" our line is? We consider the residual sum of squares (or the sum of squared errors), $RSS$: 

$$
RSS = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

where $y_i$ is the actual output value.

A large $RSS$ indicates a poorly fitted curve. Hence in order to properly fit a curve, we want to minimize $RSS$ by adjusting our weights, and we can do this with gradient descent. The idea of gradient descent is to "move around" in the solution space until we find a minimum (praying that it is close to the global minimum). So then, how do we know which direction to move in? We consider the gradient matrix, $\nabla S$:

$$
\nabla RSS = \begin{bmatrix} \frac{\partial (RSS)}{\partial w_1} \\ \frac{\partial (RSS)}{\partial w_2} \\ \vdots \\ \frac{\partial (RSS)}{\partial w_m} \end{bmatrix}
$$

$\nabla RSS$ gives us the direction to move in. So, taking $\mathbf{w} - lr * \nabla RSS$ essentially lets us descend closer to the minimum. Also, the adjustable coeffient $lr$ is there to help prevent us from exceeding the speed limit and overshooting the minimum as we descend. All thats left now is to find a way to calculate $\nabla RSS$.

Take note that the partial derivative of each weight with respect to the sum of squared residuals is given by:

$$
\frac{\partial (RSS)}{\partial w_k} = -2 \sum_{i=1}^{n} (y_i - \hat{y}_i) \cdot x_{ik}
$$

Iterating over all the weights to find each partial derivative, would be so troublesome and messy though, but I am not done with my dirty little tricks. Let's use multiplication again:

$$
\nabla RSS = \mathbf{X}^T (\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{bmatrix} - \mathbf{Y}) 
$$

Now all we have to do is implement it. Thankfully numpy exists.

In [ ]:
class lin_reg_model:
    
    def __init__(self, name, weights, steps, lr):
        self.name = name

        # ensure that steps, lr and the weights are of the right data type
        if type(steps) != int:
            raise TypeError("steps must be an int!")
        if (type(lr) != float) and (type(lr) != int):
            raise TypeError("learning rate must be an int or float!")
        if type(weights) != np.ndarray:
            raise TypeError("weights must be a matrix")
        
        self.weights = weights.astype(float) # defining the attributes
        self.steps = steps
        self.lr = lr

    
    #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
    #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    
    # defining our prediction function (y = w1*x1 + w2*x2 + ... + wn*xn)
    # note that w is a n by 1 matrix that contains our n weights
    def predict(self, X):
        if not all(isinstance(item, list) for item in X): # this ensures that we can enter a single input as a single list
            X.append(1)
            X = np.array(X)
            return float(np.matmul(X, self.weights)) 
        else: # if we are considering multiple inputs its a different story
            X = np.array(X) 
            num_rows = np.shape(X)[0] # getting the number of columns
            print(np.array([ [1] for i in range(num_rows)]))
            X = np.append(X, np.array([ [1] for i in range(num_rows)]), axis = 1) 
            return [float(i) for i in np.matmul(X, self.weights)] # convert each value in the matrix to a float before returning

    # defining a prediction function for internal calculations only
    def yhat(self, X):
        return np.matmul(X, self.weights) 
    
    # defining a function to calculate residuals
    def res(self, datapoints):
        X, Y = datapoints
        return Y - self.yhat(X) 

    # defining a function to find the sum of the square of the residuals, which we abbreviated as RSS
    def RSS(self, datapoints):
        return sum(self.res(datapoints) ** 2) 

    # defining a function for the root mean square error
    def rmse(self, datapoints):
        num_of_points = np.shape(datapoints[0])[1] # finding the number of datapoints by finding the number of rows
        return math.sqrt(self.RSS(datapoints) / num_of_points) 
    
    # defining a function dRSS to find the partial derivative of RSS with respect to each weight in self.weights
    # the values of the partial derivatives will be stored in a n by 1 matrix
    def dRSS_dw(self, datapoints):
        X, Y = datapoints
        return -2 * np.matmul(X.transpose(), self.res(datapoints)) # matrix multiplication can get us the partial derivatives easily

    def improve(self, datapoints):
        return self.weights - self.lr * self.dRSS_dw(datapoints) # return the improved self.weights

    # defining a function to train the model by repeating the improve function for each training datapoint
    # steps is the number of iterations and lr is the learning rate
    def train(self, training_datapoints, testing_datapoints = None):
        if testing_datapoints == None:
            testing_datapoints = training_datapoints

        print(f"the current root mean square error is {self.rmse(testing_datapoints)}")
        print("training model...")
        for i in range(self.steps): 
            self.weights = self.improve(training_datapoints) # set self.weights to the improved self.weights
        # print(self.weights)
        print("model trained!")
        print(f"the root mean square error is now {self.rmse(testing_datapoints)}") # seeing our improvements

## Data Munging
Before feeding the data to the linear regression model, we need it to be in a more usable format. Ideally, we want 2 lists, one for training and one for testing, where each element in each list is a datapoint.

In [3]:
# def mung(filename):
#     with open(filename) as f:
#         data = f.read().split("\n") # split each line into individual elements in the training data list
#         data.remove("") # the last element of training data is ""

#         X = [ data.split(",")[:-1] + [1] for data in data ] # extract the X values
#         Y = [ data.split(",")[-1] for data in data ] # extract the Y values
        
#         X.remove(X[0]) # purifying the data by removing the headers
#         Y.remove(Y[0]) 
        
#         X = np.array([ [ float(i) for i in j] for j in X ]) # converting all entries to floats
#         Y = np.array([ float(i) for i in Y])

#         B = np.array([ [1] for i in range(np.shape(X)[0]) ]) # creating a column of 1s for the bias

#         return (X, Y)
    
# data must be in the form where the header of the csv file is "x1, x2, ..., x3, y"
def mung(filename):
    with open(filename) as f:
        data = [ datapoint.split(",") for datapoint in f.read().split("\n")] 
        
        data.remove(data[-1])
       
        X = [ datapoint[:-1] + ["1"] for datapoint in data] # note here that we add an additional "1" to account for the bias
        Y = [ datapoint[-1] for datapoint in data ] 
        
        X.remove(X[0])
        Y.remove(Y[0])

        X = np.array(X).astype(float)
        Y = np.array(Y).astype(float)

        # print(cat_0)
        # print(cat_1)    

        return (X, Y)

## Linear Regression in Action
To test the model above, we need some training data and testing, so we will use pizza data from [pro](https://www.progml.com/) as training data. As for testing data, ChaptGpt will generate it for us. Our model will aim to predict how many pizzas(Y) need to be made based on factors(X) such as Reservations, Temperature, Tourists, Pizzas.

We start by munging the data from the csv files with our mung function and instantiating our model

In [4]:
w = np.array([1, 1, 1, 1]) # defining our weights
pizza_forecast = lin_reg_model("pizza_forecast", w, 10000, 0.00001)
training_datapoints = mung("../data/pizza_training.csv") 
testing_datapoints = mung("../data/pizza_testing.csv")

# print(training_datapoints)
# print(testing_datapoints)

Now we are ready to train the model...

In [5]:
pizza_forecast.train(training_datapoints)

'the current root mean square error is 21.644860821913362'
'training model...'
'model trained!'
'the root mean square error is now 7.11490373107488'


We can see above that there is a huge imrpovement in the root mean square error, so our model seems to be working. Finally, the moment we have been waiting for: Our pizza_forecast machine is ready! Let's do some predicting, with values from the testing dataset. Lets's say today we have 11 reservations, and the temperature is 22 degress. Furthermore, there are 8 tourists in town. How many pizzas should we make today? (The correct value is )

In [6]:
pizza_forecast.predict([11, 22, 8])

40.572885875540585

The correct value is 43 so we did infact get pretty close. 

## Logisting Regression
Now that we are done, lets build a classifier, with logistic regression. Suppose we have 2 categories, A and B. Now let's say we are given an object, and a set of inputs pertaining to that object, and we want to predict the category the object is in. Clearly, our linear regression model above won't work, so we need some new toys.

What we want is a model that gives us outputs between 0 and 1. For example, an output greater than a threshold value, say 0.5, predicts that the object is likely in category A while an output less than the threshold value could predict that the object is likely in B. One way to achieve this is to scale our linear regression line to a value between 0 and 1 using the sigmoid function:

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

Notice that $0 < \sigma(x) < 1$ for all real numbers x. If we simply consider $\sigma(y)$ where $y$ is our predicted output value, if we were to feed it to the regression model, magic happens! 

$$
\sigma(x_1, \dots, x_k) = \frac{1}{1 + e^{-w_1 x_1 + \dots + w_kx_k + b}}
$$

where $x_1, \dots, x_k$ are our model's inputs. 

An important thing to note is that if we are using the sigmoid function, (as you might have found out the hard way,) we cannot use our familiar sum of square residuals to measure the error. This is because it will give a curve that gradient descent won't like i,e. not a very smooth one.

So instead of trying to minimise the sum of squared errors, we will use the maximum log of likelihood estimation (MLE). What this does is it finds the sigmoid curve that gives us the greatest probability of observing our training data i.e the likelihood. Say we have $n$ of our datapoints predicting 1s and $m$ predicting 0s. Furthermore, supposed that when the actual outputs are 1 and 0, the model outputs $p_j \,\text{for some} \, j \in \{1, 2, \dots, n\}$ and $q_k \,\text{for some} \, k \in \{1, 2, \dots, m\}$ respectively. Then, the likelyhood, $L$, is given by:

$$
L = \prod_{i=1}^{m} \hat{y}_i \prod_{i=m+1}^{n} (1 - \hat{y}_i)
$$

However, to we want to make use of gradient descent (or ascent in this case), so we need to find a gradient matrix. However, L is not well behaved and differentiating him could lead to the destruction of our planet. Hence we consider the log of the likelihood, $\ell = log(L)$. This is is nice because it turns $L$ into a summation (instead of a multiplication), which is much easier to differentiate. Also, the set of weights that give the maximum $\ell$ also give the maximum $L$. Hence we will use the log of the likelihood:

$$
\ell = \sum_{i=1}^{m} \log(\hat{y_i}) + \sum_{i=m+1}^{n} \log(1 - \hat{y_i}) 
$$

However, if we rewrite $\ell$ as $\ell= \sum_{i=1}^{n} \left[ y_i \log(\hat{y_i}) + (1 - y_i) \log(1 - \hat{y_i}) \right]$, it will be much easier to calculate.

Now, if we take the partial derivative with respect to some weights, say $w_i$, we get:

$$
\frac{\partial \ell}{\partial w_i} = \sum_{i=1}^{n} \left[x_i \left( y_i - \hat{y_i} \right) \right]
$$

where $y_i$ is the actual value for each datapoint. 

Wow, it simplified very nicely. It looks familiar, doesn't it? Yes, we can use matrices to express the gradient matrix, $\nabla \ell$ as:

$$
\nabla \ell = \mathbf{X}^T (\mathbf{Y} - \begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{bmatrix})
$$

where the definitions of $\mathbf{Y}$ and $\mathbf{X}^T$ are identical to the above definitions.

Before we start, because we are finding the $maximum$ likelihood, we need to acscend. So we take $\mathbf{w} + lr * \nabla \ell$ (instead of $\mathbf{w} - lr * \nabla \ell$ like above).

Now we can start writing code. We will start by creating a new class that'll inherit some of the functionality from lin_reg_model.

In [7]:
class log_reg_model(lin_reg_model):
    def __init__(self, name, weights, steps, lr, threshold = 0.5):
        super().__init__(name, weights, steps, lr)
        self.threshold = threshold
    
    
    #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
    #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#


    # defining a sigmoid function
    def __sigma(self, X):
        return (1 + np.exp(-X)) ** -1

    # defining our prediction function sigma(x)
    def predict(self, X, show_yhat = False, add_weight = True):
        if not all(isinstance(item, list) for item in X) and not all(isinstance(item, type(np.array([0]))) for item in X): # this ensures that we can enter a single input as a single list
            if add_weight:
                X+=[1]
            X = np.array(X)
            Y = np.matmul(X, self.weights)
            # print(np.where(self.__sigma(Y) >= self.threshold, 1, 0))

            if show_yhat:
                print(f"yhat: {self.__sigma(Y)}")
                
            return int(np.where(self.__sigma(Y) >= self.threshold, 1, 0))
        else: # if we are considering multiple inputs its a different story
            if add_weight:
                X = np.array(X) 
                num_rows = np.shape(X)[0] # getting the number of columns
                # print(np.array([ [1] for i in range(num_rows)]))
                X = np.append(X, np.array([ [1] for i in range(num_rows)]), axis = 1) 
                
            Y = np.matmul(X, self.weights)

            if show_yhat:
                print(f"yhat: {self.__sigma(Y)}")

            return np.where(self.__sigma(Y) >= self.threshold, 1, 0) 

    # defining a prediction function for internal calculations only
    def yhat(self, X, clip = True):
        Y = np.matmul(X, self.weights)
        if clip:
            return np.clip(self.__sigma(Y), 0.000001, 0.999999) # this is to prevent log of numbers very close to 0
        else:
            return self.__sigma(Y)

    # defining a function to measure level of log of the likelihood
    def log_likelihood(self, datapoints):
        
        X, Y = datapoints
         
        log_y = np.log(self.yhat(X))
        log_1_y = np.log(1 - self.yhat(X))

        return np.sum(Y * log_y + (1 - Y) * log_1_y)

    # defining a function to measure the percentage of correct predictions
    def percentage_correct(self, datapoints):
        X, Y =  datapoints
        
        num_correct = sum(np.equal(Y, self.predict(X, add_weight = False)))
        return num_correct/X.shape[0]

    # defining a function to calculate the gradient matrix for log likelyhood
    def dl_dw(self, datapoints):
        X, Y = datapoints
        return np.matmul(np.transpose(X), Y - self.yhat(X))

    def improve(self, datapoints):
        return self.weights + self.lr * self.dl_dw(datapoints)

    # defining a function to train the model by repeating the improve function for each training datapoint
    # steps is the number of iterations and lr is the learning rate
    def train(self, training_datapoints, testing_datapoints = None):
        if testing_datapoints == None:
            testing_datapoints = training_datapoints

        print(f"the current log likelihood is {self.log_likelihood(testing_datapoints)}")
        print("training model...")

        for i in range(self.steps): 
            self.weights = self.improve(training_datapoints) # set self.weights to the improved self.weights
            # print(self.log_likelihood(training_datapoints))
        # print(self.weights)
        print("model trained!")
        print(f"the log likelihood is now {self.log_likelihood(testing_datapoints)}") # seeing our improvements


## Logistic Regression in action
To see if our logistic regression is successful, we will ask chatgpt to generate a well-behaved heart disease dataset. Based on a variety of factors like age, sex, blood pressure, cholesterol, and max heart rate, this model aims to predict whether an individual is at risk for heart disease.

If we were to use a real dataset, we would first have to clean it and have some way of dealing with outliers. Furthermore, there may be many more unseen factors not recorded in the dataset, which could greatly affect the results of our logistic regression. However, handling these potential problems is out of the scope of this project, so we will ask chatgpt for a synthetic data set to test our model.

In [8]:
training_datapoints = mung("../data/heart_disease_training.csv") 
# print(training_datapoints)
w = [ 1 for i in training_datapoints[0][0] ] # the number of inputs is the number of weights hence we just use the first input as reference
w = np.array(w)

In [9]:

classifier = log_reg_model("classifier", w, 100000, 0.000001, 0.5) 

Now we are finally ready to train the model...

In [10]:
classifier.train(training_datapoints) # train the model!

'the current log likelihood is -773.6686342444104'
'training model...'
'model trained!'
'the log likelihood is now -27.042090189966498'


However, we are faced with one more issue: how do we decide on the threshold value?

To truly obtain the best threshold value, we will need to look at the distribution, but I am lazy, so I'll just use a threshold value of 0.5.

Let's see what percentage of the testing data we get correct...

In [11]:
testing_datapoints = mung("../data/heart_disease_testing.csv")

print(f"Percentage correct without logistic regression:{classifier.percentage_correct(testing_datapoints)}")

classifier.weights = [ 1 for i in training_datapoints[0][0] ] # weights here are not improved
print(f"Percentage correct without logistic regression:{classifier.percentage_correct(testing_datapoints)}")

'Percentage correct without logistic regression:0.75'
'Percentage correct without logistic regression:0.55'


Alright it's not great but there was an improvement. However, you could probably get better results with another threshold value. 

At least we know our logistic regression is probably working.